In [86]:
%pip install accelerate>=0.26.0
%pip install datasets>=4.4.1
%pip install evaluate>=0.4.6
%pip install scikit-learn>=1.7.2
%pip install spacy>=3.8.11
%pip install torch>=2.9.1
%pip install transformers>=4.57.3

/home/dmitry/Projects/NLP/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.
/home/dmitry/Projects/NLP/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.
/home/dmitry/Projects/NLP/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.
/home/dmitry/Projects/NLP/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.
/home/dmitry/Projects/NLP/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.
/home/dmitry/Projects/NLP/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.
/home/dmitry/Projects/NLP/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


In [87]:
# Config & imports
import evaluate
import numpy as np
import torch
from datasets import load_dataset
from sklearn.metrics import classification_report
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments,
    pipeline,
)

SEED = 42
MODEL_NAME = "DeepPavlov/rubert-base-cased"
# MODEL_NAME = "Den4ikAI/rubert-large-squad"
DATASET_NAME = "Davlan/sib200"
DATASET_LANGUAGE = "rus_Cyrl"
MINIBATCH_SIZE = 64

In [88]:
# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [89]:
# Load data
train_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split="train")
validation_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split="validation")
test_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split="test")

In [90]:
# Tokenize
def tok(batch):
    return tokenizer(batch["text"], truncation=True)


tokenized_train_set = train_set.map(tok, batched=True)
tokenized_validation_set = validation_set.map(tok, batched=True)

In [91]:
# Labels mapping
list_of_categories = sorted(
    list(
        set(train_set["category"])
        | set(validation_set["category"])
        | set(test_set["category"])
    )
)
indices_of_categories = list(range(len(list_of_categories)))
n_categories = len(list_of_categories)
id2label = dict(zip(indices_of_categories, list_of_categories))
label2id = dict(zip(list_of_categories, indices_of_categories))

labeled_train_set = tokenized_train_set.add_column(
    "label", [label2id[val] for val in tokenized_train_set["category"]]
)
labeled_validation_set = tokenized_validation_set.add_column(
    "label", [label2id[val] for val in tokenized_validation_set["category"]]
)

In [92]:
# Data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, pad_to_multiple_of=8)

In [93]:
# Metric
cls_metric = evaluate.load("f1")


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return cls_metric.compute(
        predictions=predictions, references=labels, average="macro"
    )


In [94]:
# Model
classifier = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=n_categories,
    id2label=id2label,
    label2id=label2id,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [95]:
# Training
training_args = TrainingArguments(
    output_dir="rubert_sib200",
    learning_rate=2e-5,
    per_device_train_batch_size=MINIBATCH_SIZE,
    per_device_eval_batch_size=MINIBATCH_SIZE,
    gradient_accumulation_steps=2,
    num_train_epochs=12,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1",
    greater_is_better=True,
    logging_steps=50,
    warmup_ratio=0.1,
    lr_scheduler_type="linear",
    seed=SEED,
    data_seed=SEED,
    report_to=["none"],
    fp16=torch.cuda.is_available(),
    no_cuda=not torch.cuda.is_available(),
    label_smoothing_factor=0.1,
    gradient_checkpointing=True,
)

trainer = Trainer(
    model=classifier,
    args=training_args,
    train_dataset=labeled_train_set,
    eval_dataset=labeled_validation_set,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print(torch.cuda.is_available())

trainer.train()

/tmp/ipykernel_7763/631175227.py:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


True


Epoch,Training Loss,Validation Loss,F1
1,No log,1.929826,0.094265
2,No log,1.767348,0.223563
3,No log,1.583113,0.388095
4,No log,1.399473,0.619709
5,No log,1.226580,0.704877
6,No log,1.086282,0.740647
7,No log,0.985094,0.790799
8,No log,0.910318,0.816439
9,1.395700,0.869398,0.834722
10,1.395700,0.844750,0.821845


TrainOutput(global_step=72, training_loss=1.1905811892615423, metrics={'train_runtime': 50.3228, 'train_samples_per_second': 167.161, 'train_steps_per_second': 1.431, 'total_flos': 268541553746400.0, 'train_loss': 1.1905811892615423, 'epoch': 12.0})

In [96]:
# Eval
trainer.evaluate()

{'eval_loss': 0.8693979978561401,
 'eval_f1': 0.8347223922226344,
 'eval_runtime': 0.0519,
 'eval_samples_per_second': 1907.378,
 'eval_steps_per_second': 38.533,
 'epoch': 12.0}

In [97]:
clf = pipeline(
    "text-classification",
    model=classifier,
    tokenizer=tokenizer,
    device=-1,
)

# Преобразуем в список строк
validation_texts = list(validation_set["text"])
pred_val = [x["label"] for x in clf(validation_texts)]
true_val = validation_set["category"]
print("Validation report:")
print(classification_report(y_true=true_val, y_pred=pred_val, digits=4))

# Аналогично для тестового набора
test_texts = list(test_set["text"])
pred_test = [x["label"] for x in clf(test_texts)]
true_test = test_set["category"]
print("Test report:")
print(classification_report(y_true=true_test, y_pred=pred_test, digits=4))

Device set to use cuda:0


Validation report:
                    precision    recall  f1-score   support

     entertainment     0.8333    0.5556    0.6667         9
         geography     0.7778    0.8750    0.8235         8
            health     1.0000    0.7273    0.8421        11
          politics     0.8667    0.9286    0.8966        14
science/technology     0.8929    1.0000    0.9434        25
            sports     1.0000    0.9167    0.9565        12
            travel     0.6818    0.7500    0.7143        20

          accuracy                         0.8485        99
         macro avg     0.8646    0.8219    0.8347        99
      weighted avg     0.8567    0.8485    0.8460        99

Test report:
                    precision    recall  f1-score   support

     entertainment     0.9167    0.5789    0.7097        19
         geography     0.8889    0.9412    0.9143        17
            health     1.0000    0.9091    0.9524        22
          politics     0.9355    0.9667    0.9508        30
scie